In [1]:
##get data
import numpy as np
import sys
sys.path
sys.path.append('../models')
import pickle
import math
import tensorflow as tf
import gpflow
import random
from sklearn import preprocessing

In [2]:
user_index = 12
day_index = 13
baseline_indices = [i for i in range(12)]
psi =[0,6]

In [45]:
user_index = 128
day_index = 129
baseline_indices = [i for i in range(128)]
psi =[0,64]

In [3]:
##set grid
u1 = np.arange(.1,1.1,.1)
u2 = np.arange(.1,1.1,.1)
rho = np.arange(.8,1.1,.05)

In [4]:
v1 = np.arange(.1,1,.1)
v2 = np.arange(.1,2,.5)

In [5]:

noise = np.arange(5000.0,6000.0,100.0)


In [6]:
#_onehot_rm_mid
#mid_nonhot
with open('../../Downloads/processed_data_for_gridsearch_GP_rm_mid_nonhot.pkl','rb') as f:
    data = pickle.load(f)
    

In [7]:
X=data[0]
y=data[1]

In [8]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))


In [9]:
import CustomKernelStatic


In [10]:
X[900][-3:]

[-0.0, 4.0, 37.0]

In [10]:
def get_likelihood(u1,u2,s1,s2,rho,noise,X,y,rhos,users):
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(194,mysession=sess,rhos=rhos,select_users=users,baseline_indices=[i for i in range(192)],\
                  psi_indices=[0,64],user_day_index=193,user_index=192,num_data_points=len(X),initial_u1=u1,initial_u2=u2,\
                           initial_s1=s1,initial_s2=s2,initial_rho=rho,initial_noise=noise)
  
    m = gpflow.models.GPR(X,y, kern=k)
    m.initialize(session=sess)
 
    try:
        l =m.compute_log_likelihood()
        print('huzzah')
        print(l)
    except:
        l=-100
        print('not huzzah not')
        print(l)
    sess.close()
    return l
    ##get likelihood #return likelihood

In [11]:
def run():
    to_save = {}
    u1 = np.arange(.1,1.1,.1)
    u2 = np.arange(.1,1.1,.1)
    first_rhos = np.arange(.8,1.1,.05)
    v1 = np.arange(1,10,1)
    v2 = np.arange(1,10,1)
    
    noise = np.arange(400.0,600.0,100.0)
    
    with open('../../Downloads/processed_data_for_gridsearch_GP_onehot.pkl','rb') as f:
        data = pickle.load(f)
    
    X=data[0]
    y=data[1]
    #for x in X:
        #x[0]=1+random.random()/.01
   # for x in X:
       # x[6]=1+random.random()/.01
    newX = []
    for x in X:
        newX.append(np.array([float(i) for i in x]))
    newX=np.array(newX)
    
    rdayone = [x[193] for x in newX]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    users = np.array([[float(newX[i][192]==newX[j][192]) for j in range(len(X))] for i in range(len(X))])
    
    print(X.shape)
    print(y.shape)
    print(len(set(X[:,0])))
    for uone in u1:
        for utwo in u2:
            for rho in first_rhos:
                for s1 in v1:
                    for s2 in v2:
                        for n in noise:
                            likli=get_likelihood(uone,uone,s1,s2,rho,n,newX,y,rhos,users)
                             
                            key = '-'.join([str(i) for i in[uone,utwo,rho,s1,s2,n]])
                            to_save[key]=likli
                            
    return to_save

In [15]:
 #run()
    

In [12]:
def new_standardize(X,y):
    new_x = [x[:-2] for x in X]
    new_x = np.array(new_x)
    #ds = np.diag(np.random.rand(new_x.shape[0]))
    #ds = np.random.rand(new_x.shape[0],new_x.shape[1])
    #print(ds)
    #print(new_x)
    #new_x = np.add(ds,new_x)
    #feature_range=(0, 1)
    mm = preprocessing.MinMaxScaler(feature_range=(.01, 1))
    new_x =mm.fit_transform(np.array(new_x))
    to_return = np.zeros((len(X),len(X[0])))
    for i in range(len(X)):
        #temp=np.zeros(len(X[i]))
        to_return[i][:-2]=new_x[i]
        #to_return[i][0]=X[i][0]
        #to_return[i][64]=X[i][64]
        to_return[i][-2]=X[i][-2]
        to_return[i][-1]=X[i][-1]
   # mm = preprocessing.MinMaxScaler(feature_range=(50, 100))
    #y=mm.fit_transform(y)
    return [to_return,y]

In [14]:
    #X=data[0]
    #y=data[1]
    #for x in X:
    #    x[0]=1+random.random()
    #for x in X:
    #    x[6]=1+random.random()

In [18]:
newX = []
for x in X:
    newX.append(np.array([float(i) for i in x]))
newX=np.array(newX)
#newX=newX.astype(float)
newy=y

In [13]:
def get_Xy(how_many):
    X=data[0]
    y=data[1]
    
    newX = []
    for x in X:
        newX.append(np.array([float(i) for i in x]))
    newX=np.array(newX)

    newy=y
    
    perms = np.random.permutation(newX.shape[0])
    
    temp_x = np.array([newX[i] for i in perms])
    temp_y = np.array([newy[i] for i in perms])
    #temp_x,temp_y = new_standardize(temp_x,temp_y)
    return temp_x[:how_many],temp_y[:how_many]
    

In [14]:
newX,newy = get_Xy(len(X))

In [15]:

    rdayone = [x[day_index] for x in newX]
    rdaytwo = rdayone
    rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(newX))] for i in range(len(newX))])
    users = np.array([[float(newX[i][user_index]==newX[j][user_index]) for j in range(len(newX))] for i in range(len(newX))])
    

In [16]:
newX.shape

(6286, 14)

In [43]:
#newX,newy = new_standardize(newX,newy)

#newy=y

In [44]:
#newX[300]

In [138]:
newy

array([[298.],
       [  0.],
       [492.],
       ...,
       [  0.],
       [314.],
       [102.]])

In [55]:
u1[0]=42185.919130213224
u2[0]=1e-06
v1[0]=1e-06
v2[0]= 1e-06
noise[0]=90448.869339658486
rho[0]=2

In [19]:
sess.close()


In [ ]:
    sess = tf.Session()
    k =  CustomKernelStatic.CustomKernelStatic(len(X[0]),mysession=sess,rhos=rhos,select_users=users,baseline_indices=baseline_indices,\
                  psi_indices=psi,user_day_index=day_index,user_index=user_index,num_data_points=len(newX),initial_u1=u1[0],initial_u2=u2[0],\
                           initial_s1=v1[0],initial_s2=v2[0],initial_rho=rho[0],initial_noise=noise[0])
  
    m = gpflow.models.GPR(newX,newy, kern=k)
    m.initialize(session=sess)
    #m.likelihood.variance=0
    #m.likelihood.variance.trainable =False

In [ ]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)
#lik=m.compute_log_likelihood()



In [21]:
print(m.kern.sigma_u1.value)
print(m.kern.sigma_u2.value)
print(m.kern.sigma_v.value)
print(m.kern.sigma_rho.value)
print(m.kern.noise_term.value)
print(m.likelihood.variance.value)

3579.43015185706
1e-06
[[[4492.02905157    0.        ]
  [   0.         2027.39758508]]]
4.332451923484488e-24
14367.371087724912


In [48]:
print(m.kern.sigma_u1.value)
print(m.kern.sigma_u2.value)
print(m.kern.sigma_v.value)
print(m.kern.sigma_rho.value)
print(m.kern.noise_term.value)

42185.919130213224
1e-06
[[[1.e-06 0.e+00]
  [0.e+00 1.e-06]]]
2.0
63591.23048226423


In [38]:
print(m.kern.sigma_u1.value)
print(m.kern.sigma_u2.value)
print(m.kern.sigma_v.value)
print(m.kern.sigma_rho.value)
print(m.kern.noise_term.value)

32444.132304687628
1e-06
[[[1.36860511e+03 0.00000000e+00]
  [0.00000000e+00 1.00000000e-06]]]
9.24832631638378e-187
72107.18149736902


In [92]:
m.kern.sigma_u1.value

array(35636.58405931)

In [93]:
m.kern.sigma_u2.value

array(35636.58405931)

In [94]:
m.kern.sigma_v.value

array([[[1.00000000e-01, 0.00000000e+00],
        [0.00000000e+00, 4.78052777e+03]]])

In [95]:
m.kern.sigma_rho.value

array(2.)

In [96]:
m.kern.noise_term.value

array(72004.57408901)

In [24]:
get_sigma_u(3579.43015185706,1e-06,0)

array([[ 3.57943015e+03, -5.98283390e-02],
       [-5.98283390e-02,  1.00000000e-06]])

In [23]:
def get_sigma_u(u1,u2,rho):
    off_diagaonal_term = u1**.5*u2**.5*(rho-1)
    return np.array([[u1,off_diagaonal_term],[off_diagaonal_term,u2]])

In [33]:
sess.close()

In [22]:
lik

-1263332.3926229083

In [98]:
lik

-2450425.7153464314

In [23]:
lik


-1269271.1689228236

In [49]:
for x in X:
    x[0]=1



In [103]:
np.array([yy[0] for yy in y]).std()

407.30819143947593

In [91]:
for i in range(len(X[0])):
    print(i)
    print(len(set(X[:,i])))
    print('\n')

0
1


1
2


2
2


3
4


4
2


5
2


6
1


7
2


8
2


9
4


10
2


11
2


12
2


13
3


14
3


15
7


16
3


17
3


18
36


19
62




In [25]:
    m = np.eye(None)
    m = np.add(m,.1)

TypeError: 'NoneType' object cannot be interpreted as an integer